In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
url = "https://fptshop.com.vn/may-tinh-xach-tay"
driver.get(url)



In [7]:
def load_more():
    try:
        # Chờ cho nút "Xem thêm" xuất hiện trong khoảng thời gian tối đa là 10 giây
        load_more_button = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@class='cdt-product--loadmore']//a"))
        )
        # Kiểm tra xem nút "Xem thêm" có hiển thị hay không
        if load_more_button.is_displayed():
            # Nhấn nút "Xem thêm"
            load_more_button.click()
            return True
        else:
            return False
    except:
        return False

In [9]:
while load_more():
    pass
product_data = []
product_links = []
html_data = driver.page_source
soup = BeautifulSoup(html_data, "html.parser")
# Tìm tất cả các sản phẩm trên trang web
product_div_lap = soup.find_all("div", class_="cdt-product")
for div in product_div_lap:
    product_info = div.find("div", class_="cdt-product__info")
    product_name = product_info.find("h3").find("a").get("title")
    product_link = product_info.find("h3").find("a").get("href")
    product_links.append(product_link)
    price_info = None
    promo_info = None

    promo_div = div.find("div", class_="cdt-product__show-promo")
    if promo_div:
        price_info = promo_div.find("strike").text.strip()
        promo_info = promo_div.find("div", class_="progress").text.strip()
    else:
        price_info = div.find("div", class_="cdt-product__price").find("div", class_="price").text.strip()

    
    config_param = div.find("div", class_="cdt-product__config__param")
    product_specs = {}
    if config_param:  # Kiểm tra xem có thông tin cấu hình hay không
        span_tags = config_param.find_all("span")
        for span in span_tags:
            title = span.get("data-title")
            value = span.text.strip()
            product_specs[title] = value

    product_data.append({
        "Tên sản phẩm": product_name,
        "Giá sản phẩm": price_info,
        "Giá Khuyến mãi": promo_info,
        **product_specs  # Thêm thông tin cấu hình vào dictionary của sản phẩm
    })
    

In [12]:
df = pd.DataFrame(product_data)

# Lưu DataFrame vào tệp CSV
df.to_csv("test.csv", index=False)